In [2]:
import h5py
import torch
import numpy as np
from sklearn.cluster import KMeans, MiniBatchKMeans

In [3]:
def kmean(x, k=512):
    x = x.reshape([-1, 2048])
    print("feature sample:", x.shape[0])
    kmeans = MiniBatchKMeans(n_clusters=k, random_state=43, verbose=True).fit(x)
    print("clustering done")
    cluster_centers = kmeans.cluster_centers_
    print("Get centers")
    labels = kmeans.labels_

    cluster_features = [x[labels == i] for i in range(k)]
    cluster_means = [np.mean(cluster, axis=0) for cluster in cluster_features]
    print("Get mean")
    return cluster_centers, cluster_means

In [ ]:
def make_sample(path="data/nextqa/video_feature/ResNet/ResNet_I_train.h5"):
    frame_feats = []
    with h5py.File(path, 'r') as fp:
        vids = fp['vid']
        feat_key = 'CLIPL_I'
        feats = fp[feat_key]
        # print(feats.shape) #v_num, clip_num, feat_dim
        for id, (vid, feat) in enumerate(zip(vids, feats)):
            vid = vid.decode("utf-8")
            frame_feats.append(feat)

    frame_feats = np.array(frame_feats)
    k_center, k_mean = kmean(frame_feats)
    k_center = torch.from_numpy(k_center)
    k_mean = torch.tensor(k_mean)
    torch.save({"k_center": k_center, "k_mean": k_mean}, "visual_clusters.npy")

In [5]:
make_sample()

KeyError: "Unable to synchronously open object (object 'ResNet_I' doesn't exist)"

In [6]:
import h5py

path = "data/nextqa/video_feature/ResNet/ResNet_I_train.h5"  # 改成你的真实路径
with h5py.File(path, "r") as f:
    print("keys:", list(f.keys()))
    for k in f.keys():
        obj = f[k]
        try:
            print(k, "shape:", obj.shape, "dtype:", obj.dtype)
        except Exception:
            print(k, "is group, subkeys:", list(obj.keys())[:20])


keys: ['CLIPL_I', 'vid']
CLIPL_I shape: (3870, 32, 768) dtype: float32
vid shape: (3870,) dtype: object
